In [1]:
import pandas as pd

In [7]:
dc_targets = pd.read_csv('drug.target.interaction.tsv', sep='\t')
dc_targets = dc_targets[dc_targets['ORGANISM'] == 'Homo sapiens']
dc_targets = dc_targets.groupby('STRUCT_ID')['GENE'].apply(lambda x: ", ".join(x))
dc_targets

STRUCT_ID
4       KCNH2, SCN1A, SCN4A, PTGER1, CYP2D6, HTR3A, KC...
5                                         CACNA1D|CACNA1C
6                        KCNN4, CACNA1D, CACNA1F, CACNA1C
13                                                  ADRB1
21                                                   DHFR
                              ...                        
5457                                                   C3
5458                                                FOLH1
5459    MST1R, ABL1, KIT, SRC, LCK, YES1, FYN, BRAF, F...
5460                                  OPRM1, OPRK1, OPRD1
5461                                                 KRAS
Name: GENE, Length: 2339, dtype: object

In [21]:
dc_structures = pd.read_csv('structures.smiles.tsv', sep='\t', usecols=['ID', 'InChIKey']).set_index('ID').sort_index().dropna(subset=['InChIKey'])
dc_structures

,InChIKey
ID,
4,LEBVLXFERQHONN-INIZCTEOSA-N
5,ZBBHBTPTTSWHBA-DEOSSOPVSA-N
6,PVHUJELLJLJGLN-INIZCTEOSA-N
13,JRWZLRBJNMZMFE-ZDUSSCGKSA-N
21,TVZGACDUOSZQKY-LBPRGKRZSA-N
...,...
5450,AJIPIJNNOJSSQC-NYLIRDPKSA-N
5459,QADPYRIHXKWUSV-UHFFFAOYSA-N
5460,RYIDHLJADOKWFM-UHFFFAOYSA-N


In [28]:
dc = pd.concat([dc_structures, dc_targets], axis=1, join='outer').rename({'GENE': 'DC Target'}, axis=1)
dc['DC Target'] = dc['DC Target'].apply(lambda x: x.replace('|', ', ') if type(x) == str else x)
dc = dc.dropna()
dc.set_index('InChIKey', inplace=True)

In [29]:
repurposing_hub_drug_targets = pd.read_csv("../../data/primary_data/repurposing_hub_drug_targets.txt", sep="\t").drop(
    ['MOA', 'Phase'], axis=1).set_index('Name')
repurposing_hub_drug_targets = repurposing_hub_drug_targets[
    ~repurposing_hub_drug_targets.index.duplicated(keep='first')]

repurposing_hub_drugs = pd.read_csv('../../data/primary_data/repurposing_hub_drugs.txt', skiprows=9, sep='\t')
repurposing_hub_drugs['broad_id'] = repurposing_hub_drugs['broad_id'].apply(lambda x: x[:-9])
repurposing_hub_drugs = repurposing_hub_drugs.loc[:, ['broad_id', 'pert_iname', 'InChIKey']].drop_duplicates().rename(
    {'pert_iname': 'Name'}, axis=1).set_index('Name')
repurposing_hub_drugs = repurposing_hub_drugs[~repurposing_hub_drugs.index.duplicated(keep='first')]
drh = pd.concat([repurposing_hub_drug_targets, repurposing_hub_drugs], axis=1, join='inner').set_index('InChIKey')
drh = drh[~drh.index.duplicated(keep='first')]
drh

,Target,broad_id
InChIKey,,
VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...",BRD-K76022557
HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",BRD-K75516118
KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",BRD-K62353271
HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",BRD-K65856711
OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,BRD-K89787693
...,...,...
RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",BRD-A07232941
QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,BRD-K03044000
XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,BRD-K00535541


In [34]:
drh_dc_merge = pd.concat([drh, dc], axis=1, join='outer').dropna(subset='broad_id').reset_index().set_index('broad_id')

In [35]:
drh_dc_merge

,InChIKey,Target,DC Target
broad_id,,,
BRD-K76022557,VMWNQDUVQKEIOC-CYBMUJFWSA-N,"ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD3...","DRD1, DRD2, ADRA2A, HTR1A, HTR2A, HTR2B, HTR2C..."
BRD-K75516118,HJORMJIFDVBMOB-LBPRGKRZSA-N,"PDE4A, PDE4B, PDE4C, PDE4D, PDE5A",NaN
BRD-K62353271,KPYSYYIEGFHWSV-QMMMGPOBSA-N,"GABBR1, GABBR2",NaN
BRD-K65856711,HJORMJIFDVBMOB-GFCCVEGCSA-N,"PDE4B, PDE4D",NaN
BRD-K89787693,OUPXSLGGCPUZJJ-SARDKLJWSA-N,TACR1,NaN
...,...,...,...
BRD-A07232941,RVIGBTUDFAGRTQ-NSHDSACASA-N,"MTNR1A, MTNR1B",NaN
BRD-K03044000,QCLFSYYUWPUWQR-UHFFFAOYSA-N,NaN,NaN
BRD-K00535541,XJGFWWJLMVZSIG-UHFFFAOYSA-N,NaN,"NQO2, NQO2"


In [36]:
drh_dc_merge.to_csv('dc_drh_merge.csv', index=True)